In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Lambda, BatchNormalization, Activation, Dropout, concatenate
from tensorflow.keras.losses import mse
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from time import time
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.keras.losses import mse, binary_crossentropy

from IPython.display import SVG

from tensorflow.keras import backend as K
from tensorflow.python.keras.callbacks import TensorBoard

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

parallelization_factor = 10

config = tf.ConfigProto(inter_op_parallelism_threads=parallelization_factor,
                      intra_op_parallelism_threads=parallelization_factor)
sess = tf.Session(config=config)

config.gpu_options.allow_growth=True

### Load the data to train the model

In [2]:
X_tcga_no_brca = pd.read_pickle("../data/tcga_raw_no_labelled_brca_log_row_normalized.pkl")
X_brca_train = pd.read_pickle("../data/tcga_brca_raw_19036_row_log_norm_train.pkl")
y_brca_train = X_brca_train["Ciriello_subtype"]
X_brca_train.drop(['tcga_id', 'Ciriello_subtype', 'sample_id', 'cancer_type'], axis="columns", inplace=True)

X_brca_test = pd.read_pickle("../data/tcga_brca_raw_19036_row_log_norm_test.pkl")
y_brca_test = X_brca_test["subtype"]

X_brca_test.drop(['tcga_id', 'subtype', 'sample_id', 'cancer_type'], axis="columns", inplace=True)

In [3]:
classify_df = pd.DataFrame(columns=["accuracy"])
validation_set_percent = 0.1

# Prepare data to train Variational Autoencoder (merge dataframes and normalize)
X_autoencoder = pd.concat([X_brca_train, X_tcga_no_brca], sort=True)
scaler = MinMaxScaler()
X_autoencoder_scaled = pd.DataFrame(scaler.fit_transform(X_autoencoder), columns=X_autoencoder.columns)

# Scale data
X_brca_train_scaled = pd.DataFrame(scaler.transform(X_brca_train), columns=X_brca_train.columns)
X_brca_test_scaled = pd.DataFrame(scaler.transform(X_brca_test), columns=X_brca_test.columns)

X_autoencoder_scaled = X_autoencoder_scaled.reindex(sorted(X_autoencoder_scaled.columns), axis="columns")
X_brca_train_scaled = X_brca_train_scaled.reindex(sorted(X_brca_train_scaled.columns), axis="columns")
X_brca_test_scaled = X_brca_test_scaled.reindex(sorted(X_brca_test_scaled.columns), axis="columns")

X_autoencoder_val = X_autoencoder_scaled.sample(frac=validation_set_percent)
X_autoencoder_train = X_autoencoder_scaled.drop(X_autoencoder_val.index)

### Initialize variables and Hyperparameters

In [4]:
original_dim = X_autoencoder_scaled.shape[1]
input_shape = (original_dim,)
hidden_dim = 300
latent_dim = 100

batch_size = 64
epochs = 100
learning_rate = 0.001

dropout_input = 0.6
dropout_hidden = 0.6

freeze_weights=False
classifier_use_z=False
noise_input=False
reconstruction_loss='binary_crossentropy'

### Define Classes

In [5]:
class BaseVAE():
	"""docstring for BaseVAE"""
	def __init__(self):
		pass

	def initialize_model(self):
		"""
		Helper function to initialize the models
		"""
		self._build_encoder_layers()
		self._build_decoder_layers()
		self._compile_vae()
		self._compile_encoder_decoder()

	def sampling(self, args):

		"""Reparameterization trick by sampling from an isotropic unit Gaussian.
		# Arguments
		args (tensor): mean and log of variance of Q(z|X)
		# Returns
		z (tensor): sampled latent vector
		"""

		z_mean, z_log_var = args
		batch = K.shape(z_mean)[0]
		dim = K.int_shape(z_mean)[1]
		# by default, random_normal has mean = 0 and std = 1.0
		epsilon = K.random_normal(shape=(batch, dim), mean=0., stddev=1.)
		
		return z_mean + K.exp(0.5 * z_log_var) * epsilon

	def vae_loss(self, y_true, y_pred):

		# E[log P(X|z)]
		if self.rec_loss == "binary_crossentropy":
			reconstruction_loss = self.original_dim * binary_crossentropy(y_true, y_pred) # because it returns the mean cross-entropy
		elif self.rec_loss == "mse":
			reconstruction_loss = self.original_dim * mse(y_true, y_pred) # because it returns the mean cross-entropy

		# D_KL(Q(z|X) || P(z|X)); calculate in closed form as both dist. are Gaussian
		kl_loss = -0.5 * K.sum(1. + self.z_log_var_encoded - K.exp(self.z_log_var_encoded) - K.square(self.z_mean_encoded), axis=1)

		return K.mean(reconstruction_loss + kl_loss)


	def save_model(self, stacked=False):
		if(not stacked):
			model_name = "../models/rna_vae_{}_hidden_{}_emb_{}_drop_in_{}_drop_hidden_noise_input_{}.h5".format(self.intermediate_dim, self.latent_dim, self.dropout_rate_input, self.dropout_rate_hidden, self.noise_input)
		else:
			model_name = "../models/rna_vae_{}_hidden_{}_emb_{}_drop_in_{}_drop_hidden_noise_input_{}_stacked_classifier.h5".format(self.intermediate_dim, self.latent_dim, self.dropout_rate_input, self.dropout_rate_hidden, self.noise_input)
		self.vae.save_weights(model_name)

	def load_model(self, filename):
		self.vae.load_weights(filename)


In [6]:
class VAE(BaseVAE):

	"""
    Building and Training a VAE)
    Modified from:
    https://github.com/greenelab/tybalt
    """

	def __init__(self, original_dim,
						intermediate_dim=0,
						latent_dim=100,
						epochs=100,
						batch_size=50,
						learning_rate=0.01,
						dropout_rate_input=0,
						dropout_rate_hidden=0,
						dropout_decoder=True,
						freeze_weights=False,
						classifier_use_z=False,
						noise_input=True,
						rec_loss="mse",
						verbose=True):

		BaseVAE.__init__(self)

		self.original_dim = original_dim
		self.intermediate_dim = intermediate_dim
		self.latent_dim = latent_dim
		self.epochs = epochs
		self.batch_size = batch_size
		self.learning_rate = learning_rate
		self.dropout_rate_input = dropout_rate_input
		self.dropout_rate_hidden = dropout_rate_hidden
		self.dropout_decoder = dropout_decoder
		self.freeze_weights = freeze_weights
		self.classifier_use_z = classifier_use_z
		self.rec_loss = rec_loss
		self.verbose = verbose
		self.noise_input = noise_input

		self.depth = 2 if (intermediate_dim>0) else 1

		print("AUTOENCODER HAS DEPTH {}".format(self.depth))

	def _build_encoder_layers(self):
		self.inputs = Input(shape=(self.original_dim, ), name="encoder_input")

		if self.noise_input:
			noise_layer_input = GaussianNoise(1)(self.inputs)
			dropout_input = Dropout(rate=self.dropout_rate_input)(noise_layer_input)

		else:
			dropout_input = Dropout(rate=self.dropout_rate_input)(self.inputs)

		if self.depth==1:
			z_mean_dense = Dense(self.latent_dim)(dropout_input)
			z_log_var_dense = Dense(self.latent_dim)(dropout_input)

		elif self.depth==2:
			hidden_dense = Dense(self.intermediate_dim)(dropout_input)
			hidden_dense_batchnorm = BatchNormalization()(hidden_dense)
			hidden_dense_encoded = Activation("relu")(hidden_dense_batchnorm)

			dropout_encoder_hidden = Dropout(rate=self.dropout_rate_hidden)(hidden_dense_encoded)

			z_mean_dense = Dense(self.latent_dim)(dropout_encoder_hidden)
			z_log_var_dense = Dense(self.latent_dim)(dropout_encoder_hidden)

		# Latent representation layers
		z_mean_dense_batchnorm = BatchNormalization()(z_mean_dense)
		self.z_mean_encoded = Activation("relu")(z_mean_dense_batchnorm)

		z_log_var_dense_batchnorm = BatchNormalization()(z_log_var_dense)
		self.z_log_var_encoded = Activation("relu")(z_log_var_dense_batchnorm)

		# Sample z
		self.z = Lambda(self.sampling, output_shape=(self.latent_dim,), name="z")([self.z_mean_encoded, self.z_log_var_encoded])

	def _build_decoder_layers(self):

		if self.depth==1:
			self.decoder_output = Dense(self.original_dim, activation="sigmoid", name="decoder_output")
			self.outputs = self.decoder_output(self.z)

		elif self.depth==2:
			self.decoder_hidden = Dense(self.intermediate_dim, activation="relu", name="decoder_hidden")
			self.decoder_output = Dense(self.original_dim, activation="sigmoid", name="decoder_output")

			vae_decoder_hidden = self.decoder_hidden(self.z)
			if self.dropout_decoder==True:
				self.decoder_dropout = Dropout(rate=self.dropout_rate_hidden)

				decoder_hidden_dropout = self.decoder_dropout(vae_decoder_hidden)
				self.outputs = self.decoder_output(decoder_hidden_dropout)

			else:
				self.outputs = self.decoder_output(vae_decoder_hidden)

	def _compile_encoder_decoder(self):

		# Compile Encoder
		self.encoder = Model(self.inputs, [self.z_mean_encoded, self.z_log_var_encoded], name="encoder")
		'''
		# Compile Decoder
		decoder_input = Input(shape=(self.latent_dim,), name='z_sampling')

		if self.depth==1:
			x_decoded = self.decoder_output(decoder_input)
		elif self.depth==2:
			x_hidden = self.decoder_hidden(decoder_input)
			if self.dropout_decoder==True:
				x_dropout = self.decoder_dropout(x_hidden)
				x_decoded = self.decoder_output(x_dropout)
			else:
				X_decoded = self.decoder_output(x_hidden)

		self.decoder = Model(decoder_input, x_decoded, name='decoder')
		'''

	def _compile_vae(self):
		"""
		Compiles all the layers together, creating the Variational Autoencoder
		"""

		adam = optimizers.Adam(lr=self.learning_rate)
		self.vae = Model(self.inputs, self.outputs, name='vae')
		self.vae.compile(optimizer=adam, loss=self.vae_loss)

	def build_classifier(self):

		if(self.freeze_weights):
			for layer in self.vae.layers:
				layer.trainable = False

		if(self.classifier_use_z):
			self.classifier_output = Dense(5, activation="softmax", name="classifier_output")(self.z)
		else:
			self.classifier_output = Dense(5, activation="softmax", name="classifier_output")(concatenate([self.z_mean_encoded, self.z_log_var_encoded], axis=1))

		self.classifier = Model(self.inputs, self.classifier_output, name="classifier")
		
		adam = optimizers.Adam(lr=self.learning_rate)
		self.classifier.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])


	def train_vae(self, train_df, val_df, val_flag=True):
		if(val_flag):
			self.train_hist = self.vae.fit(train_df, train_df,
							shuffle=True,
							epochs=self.epochs,
							batch_size=self.batch_size,
							validation_data=(val_df, val_df),
							verbose=self.verbose)
		else:
			self.train_hist = self.vae.fit(train_df, train_df,
							shuffle=True,
							epochs=self.epochs,
							batch_size=self.batch_size,
							verbose=self.verbose)

		self.hist_dataframe = pd.DataFrame(self.train_hist.history)


	def train_stacked_classifier(self, train_df, val_df, epochs):
		self.classifier.fit(train_df=X_train, y_df=y_labels_train, epochs=epochs, verbose=self.verbose)

	def evaluate_stacked_classifier(self, X_test, y_test):
		score = self.classifier.evaluate(X_test, y_test)
		return score



In [7]:
vae = VAE(original_dim=X_autoencoder_train.shape[1], 
        intermediate_dim=hidden_dim, 
        latent_dim=latent_dim, 
        epochs=epochs, 
        batch_size=batch_size, 
        learning_rate=learning_rate, 
        dropout_rate_input=dropout_input,
        dropout_rate_hidden=dropout_hidden,
        freeze_weights=freeze_weights, 
        classifier_use_z=classifier_use_z,
        noise_input=noise_input,
        rec_loss=reconstruction_loss)

AUTOENCODER HAS DEPTH 2


In [8]:
vae.initialize_model()
vae.train_vae(train_df=X_autoencoder_train, val_df=X_autoencoder_val)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Colocations handled automatically by placer.
Train on 9094 samples, validate on 1010 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
9094/9094 [==============================] - 24s 3ms/sample - loss: 10451.6304 - val_loss: 10167.3697
Epoch 2/100
9094/9094 [==============================] - 21s 2ms/sample - loss: 10133.2945 - val_loss: 10034.6082
Epoch 3/100
9094/9094 [==============================] - 21s 2ms/sample - loss: 10073.2670 - val_loss: 9986.7663
Epoch 4/100
9094/9094 [==============================] - 20s 2ms/sample - loss: 10045.9097 - val_loss: 9948.2542
Epoch 5/100
9094/9094 [==============================] - 20s 2ms/sample - loss: 10023.1053 - val_loss: 9938.5029
Epoch 6/100
9094/9094 [==============================] - 20s 2ms/sample - loss: 10011.0791 - val_loss: 9920.6885
Epoch 7/100
9094/9094 [==============

9094/9094 [==============================] - 20s 2ms/sample - loss: 9907.2273 - val_loss: 9827.4751
Epoch 65/100
9094/9094 [==============================] - 20s 2ms/sample - loss: 9905.9882 - val_loss: 9829.5702
Epoch 66/100
9094/9094 [==============================] - 20s 2ms/sample - loss: 9906.5182 - val_loss: 9828.6991
Epoch 67/100
9094/9094 [==============================] - 20s 2ms/sample - loss: 9906.7131 - val_loss: 9828.6307
Epoch 68/100
9094/9094 [==============================] - 20s 2ms/sample - loss: 9906.7805 - val_loss: 9826.9570
Epoch 69/100
9094/9094 [==============================] - 20s 2ms/sample - loss: 9906.5885 - val_loss: 9827.6608
Epoch 70/100
9094/9094 [==============================] - 20s 2ms/sample - loss: 9905.7950 - val_loss: 9827.1824
Epoch 71/100
9094/9094 [==============================] - 20s 2ms/sample - loss: 9906.6149 - val_loss: 9830.7397
Epoch 72/100
9094/9094 [==============================] - 20s 2ms/sample - loss: 9905.0603 - val_loss: 9825.5

In [11]:
enc = OneHotEncoder()
y_labels_train = enc.fit_transform(y_brca_train.values.reshape(-1, 1))
y_labels_test = enc.fit_transform(y_brca_test.values.reshape(-1, 1))

X_train_train, X_train_val, y_labels_train_train, y_labels_train_val = train_test_split(X_brca_train_scaled, y_labels_train, test_size=0.1, stratify=y_brca_train, random_state=42)

print("BUILDING CLASSIFIER")
vae.build_classifier()

fit_hist = vae.classifier.fit(x=X_train_train, 
								y=y_labels_train_train, 
								shuffle=True, 
								epochs=100,
								batch_size=64,
								callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)],
								validation_data=(X_train_val, y_labels_train_val))

BUILDING CLASSIFIER
Train on 735 samples, validate on 82 samples
Epoch 1/100
735/735 [==============================] - 1s 2ms/sample - loss: 1.6418 - acc: 0.2612 - val_loss: 1.4102 - val_acc: 0.2317
Epoch 2/100
735/735 [==============================] - 1s 1ms/sample - loss: 1.2205 - acc: 0.5687 - val_loss: 1.0630 - val_acc: 0.6707
Epoch 3/100
735/735 [==============================] - 1s 1ms/sample - loss: 0.9828 - acc: 0.7034 - val_loss: 0.9105 - val_acc: 0.6585
Epoch 4/100
735/735 [==============================] - 1s 899us/sample - loss: 0.8545 - acc: 0.7184 - val_loss: 0.8772 - val_acc: 0.6585
Epoch 5/100
735/735 [==============================] - 1s 888us/sample - loss: 0.7495 - acc: 0.7469 - val_loss: 0.8726 - val_acc: 0.6707
Epoch 6/100
735/735 [==============================] - 1s 996us/sample - loss: 0.6892 - acc: 0.7633 - val_loss: 0.8415 - val_acc: 0.7195
Epoch 7/100
735/735 [==============================] - 1s 868us/sample - loss: 0.6352 - acc: 0.7660 - val_loss: 0.8465 

735/735 [==============================] - 1s 1ms/sample - loss: 0.1917 - acc: 0.9224 - val_loss: 0.1831 - val_acc: 0.9268
Epoch 61/100
735/735 [==============================] - 1s 989us/sample - loss: 0.2260 - acc: 0.9143 - val_loss: 0.1738 - val_acc: 0.9390
Epoch 62/100
735/735 [==============================] - 1s 969us/sample - loss: 0.2401 - acc: 0.8912 - val_loss: 0.1809 - val_acc: 0.9146
Epoch 63/100
735/735 [==============================] - 1s 957us/sample - loss: 0.2144 - acc: 0.9197 - val_loss: 0.1691 - val_acc: 0.9390
Epoch 64/100
735/735 [==============================] - 1s 928us/sample - loss: 0.1951 - acc: 0.9184 - val_loss: 0.1670 - val_acc: 0.9390
Epoch 65/100
735/735 [==============================] - 1s 916us/sample - loss: 0.2311 - acc: 0.9061 - val_loss: 0.1897 - val_acc: 0.9390
Epoch 66/100
735/735 [==============================] - 1s 927us/sample - loss: 0.1855 - acc: 0.9293 - val_loss: 0.1978 - val_acc: 0.9268
Epoch 67/100
735/735 [===========================

In [16]:
vae.classifier.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 19036)        0                                            
__________________________________________________________________________________________________
dropout (Dropout)               (None, 19036)        0           encoder_input[0][0]              
__________________________________________________________________________________________________
dense (Dense)                   (None, 300)          5711100     dropout[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 300)          1200        dense[0][0]                      
__________________________________________________________________________________________________
activation

In [34]:
 = vae.classifier.layers[2].get_weights()[0]

In [43]:
first_dense_weights[0].shape

(300,)

In [35]:
np.save('../vae_weights_analysis.npy', first_dense_weights)

In [38]:
np.save('../vae_weights_analysis_genes.npy', X_tcga_no_brca.columns.values)

In [39]:
asdad = np.load('../vae_weights_analysis.npy')

In [40]:
asdad

array([[ 0.06588272, -0.00957081,  0.15896752, ...,  0.03977008,
         0.17044045,  0.16687728],
       [-0.03123969,  0.14270912,  0.11946226, ..., -0.03190156,
        -0.02870585, -0.02252444],
       [-0.1168997 , -0.22999266,  0.03431503, ...,  0.06029829,
        -0.13417697,  0.10309508],
       ...,
       [ 0.10308177, -0.10115288,  0.02363555, ..., -0.08729688,
         0.02522375, -0.05210141],
       [-0.08821324,  0.07808172,  0.06051018, ..., -0.06800039,
         0.10620739, -0.08172274],
       [-0.19392179,  0.16963923, -0.1673271 , ...,  0.09801459,
         0.02606442, -0.14854248]], dtype=float32)

In [44]:
gene_weights_grouped = [np.sum(np.absolute(arr)) for arr in first_dense_weights]

In [46]:
len(gene_weights_grouped)

19036

In [48]:
len(X_tcga_no_brca.columns.values)

19036

In [49]:
np.save('../vae_weights_analysis_grouped_absolute.npy', gene_weights_grouped)